In [6]:
import pandas as pd, numpy as np
import os, json
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, recall_score, confusion_matrix, accuracy_score, f1_score, precision_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from skopt import BayesSearchCV, Optimizer
from skopt.space import Real, Categorical, Integer
import xgboost as xgb

In [10]:
# for experiment in experiments, set pipeline flags and run bayesian optimisation on each model
# record results in new scoreboard - are there any improvements?
# if yes, use these models for ensemble, otherwise use gridsearch acquired models for ensemble

# to dynamically toggle flags for each experiment
experiments = {'snv_FS_svmsmote':{'SNV':True, 'FEATURE_SELECT':True, 'SVMSMOTE':True}, 
               'snv_augmentedv3':{'SNV':True, 'AUGMENTEDv3':True}, 
               'augmentedv3':{'AUGMENTEDv3':True}, 
               'augmentedv3_FS':{'AUGMENTEDv3':True, 'FEATURE_SELECT':True}, 
               'bordersmote':{'BORDERSMOTE':True}, 
               'svmsmote':{'SVMSMOTE':True}, 
               'augmented_FS':{'AUGMENTED':True, 'FEATURE_SELECT':True}, 
               'augmentedv2':{'AUGMENTEDv2':True}, 
               'my_balancedv2':{'MY_BALANCEDv2':True}, 
               'kmeanssmote':{'KMEANSSMOTE':True}, 
               'augmented':{'AUGMENTED':True}, 
               'adasynsmote':{'ADASYNSMOTE':True}, 
               'snv_svmsmote':{'SNV':True, 'SVMSMOTE':True}, 
               'raw':{}, 
               'smote':{'SMOTE':True}, 
               'augmentedv2_FS':{'AUGMENTEDv2':True, 'FEATURE_SELECT':True}, 
               'feature_select':{'FEATURE_SELECT':True}, 
               'snv_FS_balanced':{'SNV':True, 'FEATURE_SELECT':True, 'BALANCED':True}, 
               'balanced':{'BALANCED':True}, 
               'snv_raw':{'SNV':True}, 
               'snv_balanced':{'SNV':True, 'BALANCED':True}, 
               'feature_selectv2':{'FEATURE_SELECTv2':True}}

for experiment in experiments:
    SNV = False
    FEATURE_SELECT = False
    FEATURE_SELECTv2 = False
    BALANCED = False
    MY_BALANCED = False
    MY_BALANCEDv2 = False
    AUGMENTED = False # oversampling of neoplasia
    AUGMENTEDv2 = False # actual augmented
    AUGMENTEDv3 = False # augmented neoplasia only
    SMOTE = False
    SVMSMOTE = False
    KMEANSSMOTE = False
    ADASYNSMOTE = False
    BORDERSMOTE = False
    
    # set flags
    try:
        SNV = experiments[experiment]['SNV']
    except:
        pass
        # print(f'SNV not in {experiment}')

    try:
        FEATURE_SELECT = experiments[experiment]['FEATURE_SELECT']
    except:
        pass
        # print(f'FEATURE_SELECT not in {experiment}')

    try:
        FEATURE_SELECTv2 = experiments[experiment]['FEATURE_SELECTv2']
    except:
        pass
        # print(f'FEATURE_SELECTv2 not in {experiment}')

    try:
        BALANCED = experiments[experiment]['BALANCED']
    except:
        pass
        # print(f'BALANCED not in {experiment}')

    try:
        AUGMENTED = experiments[experiment]['AUGMENTED']
    except:
        pass
        # print(f'AUGMENTED not in {experiment}')
    
    try:
        MY_BALANCEDv2 = experiments[experiment]['MY_BALANCEDv2']
    except:
        pass
        # print(f'MY_BALANCEDv2 not in {experiment}')
    
    try:
        AUGMENTEDv2 = experiments[experiment]['AUGMENTEDv2']
    except:
        pass
        # print(f'AUGMENTEDv2 not in {experiment}')
    
    try:
        AUGMENTEDv3 = experiments[experiment]['AUGMENTEDv3']
    except:
        pass
        # print(f'AUGMENTEDv3 not in {experiment}')

    try:
        SMOTE = experiments[experiment]['SMOTE']
    except:
        pass
        # print(f'SMOTE not in {experiment}')

    try:
        SVMSMOTE = experiments[experiment]['SVMSMOTE']
    except:
        pass
        # print(f'SVMSMOTE not in {experiment}')

    try:
        KMEANSSMOTE = experiments[experiment]['KMEANSSMOTE']
    except:
        pass
        # print(f'KMEANSSMOTE not in {experiment}')

    try:
        ADASYNSMOTE = experiments[experiment]['ADASYNSMOTE']
    except:
        pass
        # print(f'ADASYNSMOTE not in {experiment}')

    try:
        BORDERSMOTE = experiments[experiment]['BORDERSMOTE']
    except:
        pass
        # print(f'BORDERSMOTE not in {experiment}')

    # Choose dataset
    DATASET = 'raw'
    FILENAME_ACC = 'BOmetrics/raw_dataset/model_metrics_accuracy_ensemble.json'
    FILENAME_RECALL = 'BOmetrics/raw_dataset/model_metrics_recall_ensemble.json'

    if BALANCED:
        DATASET = 'balanced'
        FILENAME_ACC = 'BOmetrics/balanced_dataset/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/balanced_dataset/model_metrics_recall_ensemble.json'
        if SNV:
            DATASET = 'snv_balanced'
            FILENAME_ACC = 'BOmetrics/balanced_dataset/model_metrics_accuracy_ensemble_snv.json'
            FILENAME_RECALL = 'BOmetrics/balanced_dataset/model_metrics_recall_ensemble_snv.json'
            if FEATURE_SELECT:
                DATASET = 'snv_FS_balanced'
                FILENAME_ACC = 'BOmetrics/balanced_dataset/model_metrics_accuracy_ensemble_snv_FS.json'
                FILENAME_RECALL = 'BOmetrics/balanced_dataset/model_metrics_recall_ensemble_snv_FS.json'
    elif MY_BALANCED:
        DATASET = 'my_balanced'
        FILENAME_ACC = 'BOmetrics/my_balanced_dataset/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/my_balanced_dataset/model_metrics_recall_ensemble.json'
        if FEATURE_SELECT:
            DATASET = 'FS_my_balanced'
            FILENAME_ACC = 'BOmetrics/my_balanced_dataset/model_metrics_accuracy_ensemble_FS.json'
            FILENAME_RECALL = 'BOmetrics/my_balanced_dataset/model_metrics_recall_ensemble_FS.json'
            if SNV:
                DATASET = 'snv_FS_my_balanced'
                FILENAME_ACC = 'BOmetrics/my_balanced_dataset/model_metrics_accuracy_ensemble_snv_FS.json'
                FILENAME_RECALL = 'BOmetrics/my_balanced_dataset/model_metrics_recall_ensemble_snv_FS.json'
        elif FEATURE_SELECTv2:
            DATASET = 'FSv2_my_balanced'
            FILENAME_ACC = 'BOmetrics/my_balanced_dataset/model_metrics_accuracy_ensemble_FSv2.json'
            FILENAME_RECALL = 'BOmetrics/my_balanced_dataset/model_metrics_recall_ensemble_FSv2.json'
        elif SNV:
            DATASET = 'snv_my_balanced'
            FILENAME_ACC = 'BOmetrics/my_balanced_dataset/model_metrics_accuracy_ensemble_snv.json'
            FILENAME_RECALL = 'BOmetrics/my_balanced_dataset/model_metrics_recall_ensemble_snv.json'
    elif AUGMENTED:
        DATASET = 'augmented'
        FILENAME_ACC = 'BOmetrics/augmented_dataset/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/augmented_dataset/model_metrics_recall_ensemble.json'
        if FEATURE_SELECT:
            DATASET = 'augmented_FS'
            FILENAME_ACC = 'BOmetrics/augmented_dataset/model_metrics_accuracy_ensemble_FS.json'
            FILENAME_RECALL = 'BOmetrics/augmented_dataset/model_metrics_recall_ensemble_FS.json'
    elif AUGMENTEDv2:
        DATASET = 'augmentedv2'
        FILENAME_ACC = 'BOmetrics/augmentedv2_dataset/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/augmentedv2_dataset/model_metrics_recall_ensemble.json'
        if FEATURE_SELECT:
            DATASET = 'augmentedv2_FS'
            FILENAME_ACC = 'BOmetrics/augmentedv2_dataset/model_metrics_accuracy_ensemble_FS.json'
            FILENAME_RECALL = 'BOmetrics/augmentedv2_dataset/model_metrics_recall_ensemble_FS.json'
    elif AUGMENTEDv3:
        DATASET = 'augmentedv3'
        FILENAME_ACC = 'BOmetrics/augmentedv3_dataset/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/augmentedv3_dataset/model_metrics_recall_ensemble.json'
        if SNV:
            DATASET = 'snv_augmentedv3'
            FILENAME_ACC = 'BOmetrics/augmentedv3_dataset/model_metrics_accuracy_ensemble_snv.json'
            FILENAME_RECALL = 'BOmetrics/augmentedv3_dataset/model_metrics_recall_ensemble_snv.json'
        if FEATURE_SELECT:
            DATASET = 'augmentedv3_FS'
            FILENAME_ACC = 'BOmetrics/augmentedv3_dataset/model_metrics_accuracy_ensemble_FS.json'
            FILENAME_RECALL = 'BOmetrics/augmentedv3_dataset/model_metrics_recall_ensemble_FS.json'
    elif SMOTE:
        DATASET = 'smote'
        FILENAME_ACC = 'BOmetrics/smote/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/smote/model_metrics_recall_ensemble.json'
    elif SVMSMOTE:
        DATASET = 'svmsmote'
        FILENAME_ACC = 'BOmetrics/svmsmote/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/svmsmote/model_metrics_recall_ensemble.json'
        if FEATURE_SELECT:
            DATASET = 'FS_svmsmote'
            FILENAME_ACC = 'BOmetrics/svmsmote/model_metrics_accuracy_ensemble_FS.json'
            FILENAME_RECALL = 'BOmetrics/svmsmote/model_metrics_recall_ensemble_FS.json'
            if SNV:
                DATASET = 'snv_FS_svmsmote'
                FILENAME_ACC = 'BOmetrics/svmsmote/model_metrics_accuracy_ensemble_snv_FS.json'
                FILENAME_RECALL = 'BOmetrics/svmsmote/model_metrics_recall_ensemble_snv_FS.json'
        if SNV:
                DATASET = 'snv_svmsmote'
                FILENAME_ACC = 'BOmetrics/svmsmote/model_metrics_accuracy_ensemble_FS.json'
                FILENAME_RECALL = 'BOmetrics/svmsmote/model_metrics_recall_ensemble_FS.json'
    elif KMEANSSMOTE:
        DATASET = 'kmeanssmote'
        FILENAME_ACC = 'BOmetrics/kmeanssmote/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/kmeanssmote/model_metrics_recall_ensemble.json'
    elif ADASYNSMOTE:
        DATASET = 'adasynsmote'
        FILENAME_ACC = 'BOmetrics/adasynsmote/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/adasynsmote/model_metrics_recall_ensemble.json'
    elif BORDERSMOTE:
        DATASET = 'bordersmote'
        FILENAME_ACC = 'BOmetrics/bordersmote/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/bordersmote/model_metrics_recall_ensemble.json'
    elif MY_BALANCEDv2:
        DATASET = 'my_balancedv2'
        FILENAME_ACC = 'BOmetrics/my_balancedv2_dataset/model_metrics_accuracy_ensemble.json'
        FILENAME_RECALL = 'BOmetrics/my_balancedv2_dataset/model_metrics_recall_ensemble.json'
    elif SNV:
        DATASET = 'snv_raw'
        FILENAME_ACC = 'BOmetrics/raw_dataset/model_metrics_accuracy_snv.json'
        FILENAME_RECALL = 'BOmetrics/raw_dataset/model_metrics_recall_snv.json'
    elif FEATURE_SELECT:
        DATASET = 'feature_select'
        FILENAME_ACC = 'BOmetrics/selected_features/model_metrics_accuracy.json'
        FILENAME_RECALL = 'BOmetrics/selected_features/model_metrics_recall.json'
    elif FEATURE_SELECTv2:
        DATASET = 'feature_selectv2'
        FILENAME_ACC = 'BOmetrics/selected_features/model_metrics_accuracy2.json'
        FILENAME_RECALL = 'BOmetrics/selected_features/model_metrics_recall2.json'

    # choose dataset and set x_train, x_test, y_train, y_test
    if MY_BALANCED:
        print('my_balanced')
        # x_train
        training_data = pd.read_csv('../data/balanced_data/train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/balanced_data/train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/balanced_data/test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/balanced_data/test_label.csv', header = None)
    elif BALANCED:
        print('balanced')
        # x_train
        training_data = pd.read_csv('../data/original_data/balanced_train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/original_data/balanced_train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/balanced_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/balanced_test_label.csv', header = None)
    elif AUGMENTED:
        print('augmented')
        # x_train
        training_data = pd.read_csv('../data/augmented_data/train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/augmented_data/train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)
    elif AUGMENTEDv2:
        print('augmentedv2')
        # x_train
        training_data = pd.read_csv('../data/augmented_datav2/train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/augmented_datav2/train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)
    elif AUGMENTEDv3:
        print('augmentedv3')
        # x_train
        training_data = pd.read_csv('../data/augmented_datav3/train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/augmented_datav3/train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)
    elif MY_BALANCEDv2:
        print('my_balancedv2')
        # x_train
        training_data = pd.read_csv('../data/balancedv2_data/train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/balancedv2_data/train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)
    elif SMOTE:
        print('smote')
        # x_train
        training_data = pd.read_csv('../data/SMOTE/train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/SMOTE/train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)
    elif SVMSMOTE:
        print('svmsmote')
        # x_train
        training_data = pd.read_csv('../data/SMOTE/svm/train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/SMOTE/svm/train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)
    elif KMEANSSMOTE:
        print('kmeanssmote')
        # x_train
        training_data = pd.read_csv('../data/SMOTE/kmeans/train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/SMOTE/kmeans/train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)
    elif ADASYNSMOTE:
        print('adasynsmote')
        # x_train
        training_data = pd.read_csv('../data/SMOTE/adasyn/train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/SMOTE/adasyn/train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)
    elif BORDERSMOTE:
        print('bordersmote')
        # x_train
        training_data = pd.read_csv('../data/SMOTE/border/train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/SMOTE/border/train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)
    else:
        print('raw')
        # x_train
        training_data = pd.read_csv('../data/original_data/noExclusion_train_data.csv', header = None)
        # y_train
        training_labels = pd.read_csv('../data/original_data/noExclusion_train_label.csv', header = None)
        # x_test
        testing_data = pd.read_csv('../data/original_data/noExclusion_test_data.csv', header = None)
        # y_test
        testing_labels = pd.read_csv('../data/original_data/noExclusion_test_label.csv', header = None)

    print(f"training labels vc: \n{training_labels.value_counts()}, \ntesting labels vc: \n{testing_labels.value_counts()}")
    print(len(training_data), len(training_labels), len(testing_data), len(testing_labels))

    # type cast labels to ints
    training_labels[0] = training_labels[0].astype(int)
    # testing_labels
    testing_labels[0] = testing_labels[0].astype(int)

    # encode labels, using sklearn, to pass to xgboost
    # this code was inspired by the snippet from:
    # https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
    le = LabelEncoder()
    # fit the classes to the encoder and transform labels to be 0,1,2
    training_labels = le.fit_transform(training_labels[0].to_list())
    testing_labels = le.fit_transform(testing_labels[0].to_list())

    # bayessearch results
    gs_results = {}
    print(np.unique(training_labels))

    # FEATURE SELECT
    # RUN THIS TO APPLY FEATURE SELECTION TO TRAINING DATA
    if FEATURE_SELECT == True:
        ADD_POSSIBLE_FIGURES = True

        # figures contain features from (figure_num*4)+1 
        selected_figures = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,
                            21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,
                            39,40,41,42,43,44,45,46,47,48,49,50,51,52]

        # figures that MAY be decent - noisy but different peaks
        possible_figures = [53,54,56,57,58,59,60,61,62,63,64,65]

        # decent features - eyeballed

        # generate set of selected features
        selected_features = []

        for figure_num in selected_figures:
            for i in range(0,4):
                # print(f"figure: {figure_num}, feature: feature_{(figure_num*4)+i}")
                selected_features.append((figure_num*4)+i)

        # to add possible features
        if ADD_POSSIBLE_FIGURES == True:
            for figure_num in possible_figures:
                for i in range(0,4):
                    # print(f"figure: {figure_num}, feature: feature_{(figure_num*4)+i}")
                    selected_features.append((figure_num*4)+i)
                    
        training_data = training_data[selected_features]
        testing_data = testing_data[selected_features]
    else:
        print('no feature selection')

    # FSv2
    if FEATURE_SELECTv2:    
        # selected features round 2
        selected_features = []
        a = np.arange(30,85).tolist()
        b = np.arange(203,235).tolist()

        selected_features = np.concatenate([a,b]).tolist()
        print(selected_features)
        # (Cena, 2018)
        training_data = training_data[selected_features]
        testing_data = testing_data[selected_features]
    else:
        print('no FSv2')

    # SNV
    # apply SNV to training data - inspired by code from my ML CW
    # (Hamzah Hafejee, 2022, COMP3611_Coursework_Assessment.ipynb, Comp 3611, University of Leeds)
    # (Sklearn, 2023)
    if SNV == True:
        print(len(training_data))
        # fit to training data
        scaler = StandardScaler().fit(training_data)
        training_data = scaler.transform(training_data)
        testing_data = scaler.transform(testing_data)
        print("After: \n", len(training_data))
        len(training_data)
    else:
        print('no SNV standardisation')

    

svmsmote
training labels vc: 
1    257
2    257
3    257
dtype: int64, 
testing labels vc: 
2.0    63
3.0    58
1.0    22
dtype: int64
771 771 143 143
[0 1 2]
no FSv2
771
After: 
 771
augmentedv3
training labels vc: 
3    356
2    257
1    137
dtype: int64, 
testing labels vc: 
2.0    63
3.0    58
1.0    22
dtype: int64
750 750 143 143
[0 1 2]
no feature selection
no FSv2
750
After: 
 750
augmentedv3
training labels vc: 
3    356
2    257
1    137
dtype: int64, 
testing labels vc: 
2.0    63
3.0    58
1.0    22
dtype: int64
750 750 143 143
[0 1 2]
no feature selection
no FSv2
no SNV standardisation
augmentedv3
training labels vc: 
3    356
2    257
1    137
dtype: int64, 
testing labels vc: 
2.0    63
3.0    58
1.0    22
dtype: int64
750 750 143 143
[0 1 2]
no FSv2
no SNV standardisation
bordersmote
training labels vc: 
1    257
2    257
3    257
dtype: int64, 
testing labels vc: 
2.0    63
3.0    58
1.0    22
dtype: int64
771 771 143 143
[0 1 2]
no feature selection
no FSv2
no SNV sta

In [65]:
type(training_data)
training_data.shape

(767, 433)

In [11]:
# TO USE AVG RECALL AS METRIC FOR GS
# (gunes, 2019)
gs_recall = make_scorer(recall_score, average='macro')
DATASET

'feature_selectv2'

In [21]:
# make CART classifier
clf_cart = tree.DecisionTreeClassifier(criterion="gini", random_state=1)
# find optimal parameter values for CART
params = {
    'max_depth': [None, 5, 10, 15, 20, 25, 30, 35, 40], # control overfitting,
    'max_features': [None, 'sqrt', 'log2'] # performance 
}

if DATASET == 'raw':
    # raw dataset
    params = {
        'max_depth': [None], # control overfitting,
        'max_features': ['log2'] # performance 
    }
elif DATASET == 'feature_select':
    # selected + possible features
    params = {
        'max_depth': [5], # control overfitting,
        'max_features': [None] # performance 
    }
elif DATASET == 'feature_selectv2':
    # feature select v2
    params = {'max_depth': [None], 'max_features': ['sqrt']} 
elif DATASET == 'snv_raw':
    # SNV + raw
    params = {'max_depth': [None], 'max_features': ['log2']} 
elif DATASET == 'balanced':
    params = {'max_depth': [10], 'max_features': ['sqrt']} 
elif DATASET == 'snv_balanced':
    params = {'max_depth': [10], 'max_features': ['sqrt']} 

elif DATASET == 'my_balanced':
    params = {'max_depth': [None], 'max_features': ['sqrt']} 

elif DATASET == 'FS_my_balanced':
    params = {'max_depth': [None], 'max_features': ['sqrt']} 
    
elif DATASET == 'FSv2_my_balanced':
    params = {'max_depth': [10], 'max_features': ['log2']} 
    
elif DATASET == 'snv_my_balanced':
    params = {'max_depth': [None], 'max_features': ['sqrt']} 

elif DATASET == 'snv_FS_my_balanced':
    params = {'max_depth': [5], 'max_features': [None]} 
   
elif DATASET == 'augmented':
    params = {'max_depth': [10], 'max_features': ['log2']} 

elif DATASET == 'augmented_FS':
    params = {'max_depth': [10], 'max_features': ['sqrt']} 

elif DATASET == 'smote':
    params = {'max_depth': [15], 'max_features': ['sqrt']} 

elif DATASET == 'augmentedv3':
    params = {'max_depth': [None], 'max_features': ['log2']}
    
elif DATASET == 'snv_augmentedv3':
    params = {'max_depth': [None], 'max_features': ['log2']}
 
elif DATASET == 'snv_FS_my_balanced':
    params = {'max_depth': [10], 'max_features': [None]} 

elif DATASET == 'augmentedv3_FS':
    params = {'max_depth': [None], 'max_features': ['log2']}

elif DATASET == 'kmeanssmote':
    params = {'max_depth': [10], 'max_features': ['sqrt']} 
    
elif DATASET == 'svmsmote':
    params = {'max_depth': [None], 'max_features': ['log2']}
    
elif DATASET == 'adasynsmote':
    params = {'max_depth': [None], 'max_features': ['sqrt']} 
     
elif DATASET == 'bordersmote':
    params = {'max_depth': [None], 'max_features': ['sqrt']} 
     
elif DATASET == 'snv_svmsmote':
    params = {'max_depth': [None], 'max_features': ['log2']}
    
elif DATASET == 'snv_FS_svmsmote':
    params = {'max_depth': [10], 'max_features': [None]} 
    
grid_search = GridSearchCV(clf_cart, params, scoring='accuracy', cv=5)
grid_search.fit(training_data, np.ravel(training_labels))
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_cart = grid_search.best_estimator_
gs_results['CART'] = {'accuracy':best_score, 'params':best_params}

params = {'max_depth': None, 'max_features': 'sqrt'} 
best_score: 0.7465446224256294


In [19]:
# bayes search experiment (Skopt, 2017)
clf_cart = tree.DecisionTreeClassifier(criterion="gini", random_state=1)
# find optimal parameter values for CART
NoneList = [None]
NumList = range(1,101)
for num in NumList:
    NoneList.append(num)

params = {
    'max_depth': Categorical(NoneList), # control overfitting,
    'max_features': Categorical([None, 'sqrt', 'log2']) # performance 
}
   
bayes_search = BayesSearchCV(clf_cart, params, scoring='accuracy', cv=5, n_iter=100, random_state=1)
_ = bayes_search.fit(training_data, np.ravel(training_labels))
best_params = bayes_search.best_params_
best_score = bayes_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_cart = bayes_search.best_estimator_
gs_results['CART'] = {'accuracy':best_score, 'params':best_params}

c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


params = OrderedDict([('max_depth', 12), ('max_features', None)]) 
best_score: 0.7710450038138826


In [26]:
# make Gaussian Naive Bayes classifier
clf_nb = GaussianNB()
params = {
    # use log-uniform for quicker convergence (Lewinson, 2022)
    'var_smoothing':Real(1e-20, 0.1, prior='log-uniform') # from less smoothing to more aggressive smoothing
}
bayes_search = BayesSearchCV(clf_nb, params, scoring='accuracy', cv=5, n_iter=100, random_state=1)
_ = bayes_search.fit(training_data, np.ravel(training_labels))
best_params = bayes_search.best_params_
best_score = bayes_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_nb = bayes_search.best_estimator_
gs_results['GNB'] = {'accuracy':best_score, 'params':best_params}

c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.


params = OrderedDict([('var_smoothing', 2.2400352698102787e-07)]) 
best_score: 0.6294126620900077


In [27]:
# make k-Nearest Neighbours classifier
clf_knn = KNeighborsClassifier(n_jobs=-1) # use all processes for parellelisation
params = {
    'n_neighbors': Integer(1,15)
}
bayes_search = BayesSearchCV(clf_knn, params, scoring='accuracy', cv=5, n_iter=100, random_state=1)
_ = bayes_search.fit(training_data, np.ravel(training_labels))
best_params = bayes_search.best_params_
best_score = bayes_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_knn = bayes_search.best_estimator_
gs_results['kNN'] = {'accuracy':best_score, 'params':best_params}

c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.


params = OrderedDict([('n_neighbors', 6)]) 
best_score: 0.7919908466819223


In [28]:
# make SVM-RBF classifier
clf_svmrbf = SVC(kernel='rbf', random_state=1)
params = {
    'C': Real(0.1,5000, prior='log-uniform'), # high to low regularisation strength
    'gamma' : Categorical(['scale', 'auto']), # need to research this parameter more
}

bayes_search = BayesSearchCV(clf_svmrbf, params, scoring='accuracy', cv=5, n_iter=100, random_state=1)
_ = bayes_search.fit(training_data, np.ravel(training_labels))
best_params = bayes_search.best_params_
best_score = bayes_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_svmrbf = bayes_search.best_estimator_
gs_results['SVM-RBF'] = {'accuracy':best_score, 'params':best_params}

c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.


params = OrderedDict([('C', 4401.093805632649), ('gamma', 'scale')]) 
best_score: 0.8444241037376049


In [36]:
# make SVM linear classifier
clf_lin = SVC(kernel='linear', random_state=1)
params = {
    # the different SVM kernels have different C values as I adjusetd it after seeing
    # how previous experiments went and where they most commonly performed best
    'C': Real(0.05, 1000, prior='log-uniform'), # high to low regularisation strength
    'gamma' : Categorical(['scale', 'auto']), # need to research this parameter more
    # 'gamma' : [], # need to research this parameter more - remember i started off 
    # testing each param separately to lower range of values for each 
}

bayes_search = BayesSearchCV(clf_lin, params, scoring='accuracy', cv=5, n_iter=100, random_state=1)
_ = bayes_search.fit(training_data, np.ravel(training_labels))
best_params = bayes_search.best_params_
best_score = bayes_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_svmlin = bayes_search.best_estimator_
gs_results['SVM-Lin'] = {'accuracy':best_score, 'params':best_params}

params = OrderedDict([('C', 0.7453816560797706), ('gamma', 'auto')]) 
best_score: 0.7429595728451563


In [38]:
# make svm sigmoidal classifier
clf_sig = SVC(kernel='sigmoid', random_state=1)
params = {
    'C': Real(0.00001, 100, prior='log-uniform'), # high to low regularisation strength
    'gamma' : Categorical(['scale', 'auto']), # need to research this parameter more
    # 'gamma' : [], # need to research this parameter more
}
    
bayes_search = BayesSearchCV(clf_sig, params, scoring='accuracy', cv=5, n_iter=100, random_state=1)
_ = bayes_search.fit(training_data, np.ravel(training_labels))
best_params = bayes_search.best_params_
best_score = bayes_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_svmsig = bayes_search.best_estimator_
gs_results['SVM-Sig'] = {'accuracy':best_score, 'params':best_params}

c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.


params = OrderedDict([('C', 0.10047028375600094), ('gamma', 'auto')]) 
best_score: 0.6415713196033562


In [39]:
# make xgboost classifier (Piotr Płoński, 2021)
clf_xgb = xgb.XGBClassifier(random_state = 1)

params = {
    'n_estimators': Integer(10, 1000, prior='log-uniform'), # no. boosting rounds
    'max_depth': Integer(1,50) # control overfitting
}

bayes_search = BayesSearchCV(clf_xgb, params, scoring='accuracy', cv=5, n_iter=100, random_state=1)
_ = bayes_search.fit(training_data, np.ravel(training_labels))
best_params = bayes_search.best_params_
best_score = bayes_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_xgb = bayes_search.best_estimator_
gs_results['XGB'] = {'accuracy':best_score, 'params':best_params}

c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.


params = OrderedDict([('max_depth', 9), ('n_estimators', 59)]) 
best_score: 0.8077955758962624


In [42]:
# make adaboost classifier
clf_ada = AdaBoostClassifier(random_state=1)
params = {
    'n_estimators': Integer(10,1000,prior='log-uniform'),
    'learning_rate': Real(0.001, 10, prior='log-uniform') # weight applied to each clf at each boosting iteration
}

bayes_search = BayesSearchCV(clf_ada, params, scoring='accuracy', cv=5, n_iter=100, random_state=1)
_ = bayes_search.fit(training_data, np.ravel(training_labels))
best_params = bayes_search.best_params_
best_score = bayes_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_ada = bayes_search.best_estimator_
gs_results['ADA'] = {'accuracy':best_score, 'params':best_params}

c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


params = OrderedDict([('learning_rate', 0.002923553521649806), ('n_estimators', 222)]) 
best_score: 0.6643325705568268


In [44]:
# make Logistic Regressor
clf_lr = LogisticRegression(random_state=1, max_iter=1000)
params = {
    'penalty': ['l1', 'l2'], # type of regularisation 
    'C': [0.1, 1, 10, 100], # regularisation strength
    'solver': ['liblinear', 'saga', 'lbfgs', 'newton-cg'] # approach to finding best weights
}

print(DATASET)

if DATASET == 'raw':
    # raw dataset
    params = {
        'penalty': ['l2'], # type of regularisation 
        'C': [0.1], # regularisation strength
        'solver': ['lbfgs'] # approach to finding best weights
    }
elif DATASET == 'feature_select':
    # selected + possible features
    params = {
        'penalty': ['l2'], # type of regularisation 
        'C': [0.1], # regularisation strength
        'solver': ['newton-cg'] # approach to finding best weights
    }
elif DATASET == 'feature_selectv2':
    # FSv2
    params = {'C': [1], 'penalty': ['l2'], 'solver': ['lbfgs']} 
elif DATASET == 'snv_raw':
    # SNV + raw
    params = {'C': [0.1], 'penalty': ['l2'], 'solver': ['lbfgs']} 
elif DATASET == 'balanced':
    params = {'C': [100], 'penalty': ['l2'], 'solver': ['saga']} 
elif DATASET == 'snv_balanced':
    params = {'C': [0.1], 'penalty': ['l2'], 'solver': ['saga']} 

elif DATASET == 'my_balanced':
    params = {'C': [10], 'penalty': ['l2'], 'solver': ['saga']} 
    
elif DATASET == 'FS_my_balanced':
    params = {'C': [10], 'penalty': ['l2'], 'solver': ['saga']} 
    
elif DATASET == 'FSv2_my_balanced':
    params = {'C': [10], 'penalty': ['l2'], 'solver': ['saga']} 
    
elif DATASET == 'snv_my_balanced':
    params = {'C': [1], 'penalty': ['l1'], 'solver': ['saga']} 
    
elif DATASET == 'snv_FS_my_balanced':
    params = {'C': [1], 'penalty': ['l2'], 'solver': ['saga']} 
    
elif DATASET == 'augmented':
    params = {'C': [10], 'penalty': ['l2'], 'solver': ['lbfgs']} 
    
elif DATASET == 'augmented_FS':
    params = {'C': [100], 'penalty': ['l1'], 'solver': ['liblinear']} 

elif DATASET == 'smote':
    params = {'C': [10], 'penalty': ['l2'], 'solver': ['saga']} 

elif DATASET == 'augmentedv3':
    params = {'C': [100], 'penalty': ['l1'], 'solver': ['liblinear']} 
    
elif DATASET == 'snv_augmentedv3':
    params = {'C': [10], 'penalty': ['l2'], 'solver': ['lbfgs']} 
    
elif DATASET == 'snv_FS_my_balanced':
    params = {'C': [10], 'penalty': ['l1'], 'solver': ['saga']} 

elif DATASET == 'augmentedv3_FS':
    params = {'C': [100], 'penalty': ['l1'], 'solver': ['liblinear']} 
    
elif DATASET == 'svmsmote':
    params = {'C': [100], 'penalty': ['l2'], 'solver': ['lbfgs']} 
    
elif DATASET == 'kmeanssmote':
    params = {'C': [10], 'penalty': ['l2'], 'solver': ['lbfgs']} 
    
elif DATASET == 'adasynsmote':
    params = {'C': [100], 'penalty': ['l1'], 'solver': ['liblinear']} 
    
elif DATASET == 'bordersmote':
    params = {'C': [100], 'penalty': ['l1'], 'solver': ['liblinear']} 
    
elif DATASET == 'snv_svmsmote':
    params = {'C': [100], 'penalty': ['l2'], 'solver': ['lbfgs']} 
    
elif DATASET == 'snv_FS_svmsmote':
    params = {'C': [100], 'penalty': ['l2'], 'solver': ['lbfgs']} 
    
grid_search = GridSearchCV(clf_lr, params, scoring='accuracy', cv=10)
grid_search.fit(training_data, np.ravel(training_labels))
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
gs_results['LR'] = {'accuracy':best_score, 'params':best_params}


feature_selectv2
params = {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'} 
best_score: 0.7413188142770719


In [ ]:
print(f"params = {best_params} \nbest_score: {best_score}")
best_lr = bayes_search.best_estimator_

params = {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'} 
best_score: 0.7413188142770719


In [46]:
# make Random Forest classifier
clf_rf = RandomForestClassifier(random_state=1)
NoneList = [None]
NumList = range(1,50)
for num in NumList:
    NoneList.append(num)
    
params = {
  'n_estimators': Integer(10,300,prior='log-uniform'),
  'max_depth': Categorical(NoneList),
  "max_features" : Categorical(NoneList)
}

bayes_search = BayesSearchCV(clf_rf, params, scoring='accuracy', cv=5, n_iter=100, random_state=1)
_ = bayes_search.fit(training_data, np.ravel(training_labels))
best_params = bayes_search.best_params_
best_score = bayes_search.best_score_
print(f"params = {best_params} \nbest_score: {best_score}")
best_rf = bayes_search.best_estimator_
gs_results['RF'] = {'accuracy':best_score, 'params':best_params}

params = OrderedDict([('max_depth', 49), ('max_features', 33), ('n_estimators', 25)]) 
best_score: 0.8164912280701755


AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [48]:
best_rf = bayes_search.best_estimator_
gs_results['RF'] = {'accuracy':best_score, 'params':best_params}
gs_results['RF']

{'accuracy': 0.8164912280701755,
 'params': OrderedDict([('max_depth', 49),
              ('max_features', 33),
              ('n_estimators', 25)])}

In [49]:
# ensemble model (Sklearn, 2014), (Sklearn, 2023)
ensemble = VotingClassifier(estimators=[
    ('rf', best_rf), 
    ('knn', best_knn), 
    ('xgb', best_xgb), 
    ('svmrbf', best_svmrbf), 
    ('nb', best_nb)],
    voting='hard',
    n_jobs=-1)

ensemble.fit(training_data, training_labels)

accuracy = ensemble.score(testing_data, testing_labels)
predictions = ensemble.transform(testing_data)
gs_results['Ensemble'] = {'accuracy':accuracy}
accuracy

0.7692307692307693

In [50]:
# sorted GS models
print(gs_results)
gs_sorted_models = dict(sorted(gs_results.items(), key=lambda item: item[1]['accuracy'], reverse=True))
print(gs_sorted_models.keys())

{'CART': {'accuracy': 0.7465446224256294, 'params': {'max_depth': None, 'max_features': 'sqrt'}}, 'GNB': {'accuracy': 0.6294126620900077, 'params': OrderedDict([('var_smoothing', 2.2400352698102787e-07)])}, 'kNN': {'accuracy': 0.7919908466819223, 'params': OrderedDict([('n_neighbors', 6)])}, 'SVM-RBF': {'accuracy': 0.8444241037376049, 'params': OrderedDict([('C', 4401.093805632649), ('gamma', 'scale')])}, 'SVM-Lin': {'accuracy': 0.7429595728451563, 'params': OrderedDict([('C', 0.7453816560797706), ('gamma', 'auto')])}, 'SVM-Sig': {'accuracy': 0.6415713196033562, 'params': OrderedDict([('C', 0.10047028375600094), ('gamma', 'auto')])}, 'XGB': {'accuracy': 0.8077955758962624, 'params': OrderedDict([('max_depth', 9), ('n_estimators', 59)])}, 'ADA': {'accuracy': 0.6643325705568268, 'params': OrderedDict([('learning_rate', 0.002923553521649806), ('n_estimators', 222)])}, 'LR': {'accuracy': 0.7413188142770719, 'params': {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}}, 'RF': {'accuracy': 0.81649

In [57]:
DATASET = 'feature_selectv2'
FILENAME_ACC = 'BOmetrics/selected_features/model_metrics_accuracy2.json'
FILENAME_RECALL = 'BOmetrics/selected_features/model_metrics_recall2.json'

In [58]:
# evaluate models (Sklearn, 2023)
# model_metrics = {'accuracy', 'recall', 'precision', 'F1-score', 'ROC-AUC'}
model_metrics = {}

# all of the models
models = [best_cart, best_rf, best_lr, best_nb, best_knn, best_svmrbf, best_svmlin, best_svmsig, best_xgb, best_ada, ensemble]
model_names = ['CART', 'RF', 'LR', 'GNB', 'kNN', 'SVM-RBF', 'SVM-Lin', 'SVM-Sig', 'XGB', 'ADA', 'Ensemble']
i=0
for model in models:
    # train on test set
    predicted = model.predict(testing_data)
    # generate cm against test labels
    cm = confusion_matrix(testing_labels, predicted)
    # print(cm)
    accuracy = accuracy_score(testing_labels, predicted)
    recall = recall_score(testing_labels, predicted, average=None)
    precision = precision_score(testing_labels, predicted, average=None)
    f1 = f1_score(testing_labels, predicted, average=None)

    try:
        predicted_prob = model.predict_proba(testing_data)
        roc = roc_auc_score(testing_labels, predicted_prob, average=None, multi_class='ovr') 
        # print(accuracy, recall, precision, f1, roc)
        model_metrics[model_names[i]] = {
                                            'accuracy':accuracy, 
                                            'recall':{
                                                1:recall[0], 
                                                2:recall[1], 
                                                3:recall[2]
                                            },
                                            'precision':{
                                                1:precision[0], 
                                                2:precision[1], 
                                                3:precision[2]
                                            },
                                            'f1_score':{
                                                1:f1[0], 
                                                2:f1[1], 
                                                3:f1[2]
                                            },
                                            'ROC-AUC':{
                                                1:roc[0], 
                                                2:roc[1], 
                                                3:roc[2]
                                            }
        }
    except:
        print(f"can't predict class probilities for {model_names[i]}")
        # print(accuracy, recall, precision, f1)
        model_metrics[model_names[i]] = {
                                            'accuracy':accuracy, 
                                            'recall':{
                                                1:recall[0], 
                                                2:recall[1], 
                                                3:recall[2]
                                            },
                                            'precision':{
                                                1:precision[0], 
                                                2:precision[1], 
                                                3:precision[2]
                                            },
                                            'f1_score':{
                                                1:f1[0], 
                                                2:f1[1], 
                                                3:f1[2]
                                            }
        }
    i+=1

# (Gern Blanston, 2009)- sort by neoplasia recall
sorted_metrics = dict(sorted(model_metrics.items(), key=lambda item: item[1]['recall'][3], reverse=True))
# (holys, 2013)
with open(FILENAME_RECALL, 'w+') as fp:
    json.dump(sorted_metrics, fp)

# redo but sort by accuracy
# (Gern Blanston, 2009)
sorted_metrics_acc = dict(sorted(model_metrics.items(), key=lambda item: item[1]['accuracy'], reverse=True))
# (holys, 2013)
with open(FILENAME_ACC, 'w+') as fp:
    json.dump(sorted_metrics_acc, fp)
model_metrics
sorted_metrics
FILENAME_ACC, FILENAME_RECALL
 

can't predict class probilities for SVM-RBF
can't predict class probilities for SVM-Lin
can't predict class probilities for SVM-Sig
can't predict class probilities for Ensemble


c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sc19mhh\Desktop\Hamzah\Uni\CompSci\ThirdYear\FYP\MyCode\ML-cancer-detection\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


('BOmetrics/selected_features/model_metrics_accuracy2.json',
 'BOmetrics/selected_features/model_metrics_recall2.json')

In [59]:
# print highest acc models from gridsearch
print(f"gs_sorted_models (acc): \n{gs_sorted_models.keys()}\n")

# highest acc models from test set
print(f"sorted models (acc): \n{sorted_metrics_acc.keys()}\n")

# highest recall from test set
print(f"sorted models (recall): \n{sorted_metrics.keys()}")


gs_sorted_models (acc): 
dict_keys(['SVM-RBF', 'RF', 'XGB', 'kNN', 'Ensemble', 'CART', 'SVM-Lin', 'LR', 'ADA', 'SVM-Sig', 'GNB'])

sorted models (acc): 
dict_keys(['SVM-RBF', 'Ensemble', 'XGB', 'kNN', 'CART', 'RF', 'LR', 'SVM-Lin', 'GNB', 'ADA', 'SVM-Sig'])

sorted models (recall): 
dict_keys(['GNB', 'SVM-RBF', 'CART', 'kNN', 'Ensemble', 'XGB', 'LR', 'RF', 'SVM-Lin', 'SVM-Sig', 'ADA'])


In [60]:
DATASET

'feature_selectv2'

In [64]:
# (Neekhara, 2019)
import json
# DATASET = 'snv_svmsmote'

# function to add to JSON
def write_json(new_data, filename='BOmetrics/scoreboard.json'):
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside emp_details
        file_data[DATASET] = (new_data)
        file_data = dict(sorted(file_data.items(), key=lambda item: item[1]['all']['accuracy'], reverse=True))
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)

# calculate avg accuracy and recall
accuracy = 0
recall = 0
count = 0
for model in model_names:
    # print(model_metrics[model]['accuracy'])
    accuracy += model_metrics[model]['accuracy']
    recall += model_metrics[model]['recall'][3]
    count +=1

t6acc = 0
t6rec = 0
count2 = 0
for key in sorted_metrics_acc:
    if count2 ==6:
        break
    t6acc += sorted_metrics_acc[key]['accuracy']
    t6rec += sorted_metrics_acc[key]['recall'][3]
    count2 +=1

avg = {'all' : {'accuracy': accuracy/count, 'neoplasia recall': recall/count},
       'top6' : {'accuracy': t6acc/count2, 'recall':t6rec/count2}}
print(accuracy, recall, count)
print(t6acc, t6rec, count2)
if DATASET != 'test':
    write_json(avg)
else:
    print(DATASET)
avg



7.7832167832167825 5.258620689655172 11
4.538461538461538 3.5172413793103448 6


{'all': {'accuracy': 0.7075651621106166,
  'neoplasia recall': 0.47805642633228834},
 'top6': {'accuracy': 0.7564102564102564, 'recall': 0.5862068965517241}}

# observations on raw dataset
XGBoost and ADAboost seem to have really overfit, because they severely underperform on unseen test data, compared to the accuracies they were achieving with gridsearch. IGNORE THIS: it is just because the test labels were not normalised!

Although GNB has higher recall for neoplasia than kNN, kNN seems to be the best classifier overall. While GNB has highest recall for neoplasia, has 3rd lowest accuracy.

Top models based on accuracy, from gridsearch, were RF, kNN, XGB, SVM-RBF, CART. Top models based on accuracy, from test set, were RF, kNN, SVM-RBF, CART, SVM-Lin. Therefore, RF, kNN, SVM-RBF, CART seem to perform well, in terms of accuracy, and don't seem to produce drastically different results with the test set, suggesting there isn't much overfitting

gs_sorted_models (acc): 
(['RF', 'kNN', 'XGB', 'SVM-RBF', 'CART', 'LR', 'SVM-Lin', 'ADA', 'SVM-Sig', 'GNB'])

sorted models (acc): 
(['RF', 'XGB', 'kNN', 'SVM-RBF', 'CART', 'SVM-Lin', 'LR', 'SVM-Sig', 'GNB', 'ADA'])

sorted models (recall): 
(['GNB', 'kNN', 'CART', 'RF', 'SVM-Lin', 'XGB', 'LR', 'SVM-RBF', 'SVM-Sig', 'ADA'])

# observations on feature selected dataset
Some models decreased in performance, some increased, with largest increase being 6% increase in accuracy for SVM-Lin model. But overall, not worth, since the max accuracy of any of the models was lower than without feature selection. Maybe better feature selection is needed - an analytical solution rather than eyeball

gs_sorted_models (acc): 
(['RF', 'SVM-RBF', 'kNN', 'XGB', 'CART', 'SVM-Lin', 'LR', 'ADA', 'GNB', 'SVM-Sig'])

sorted models (acc): 
(['RF', 'XGB', 'kNN', 'SVM-RBF', 'CART', 'LR', 'SVM-Lin', 'GNB', 'ADA', 'SVM-Sig'])

sorted models (recall): 
(['GNB', 'SVM-RBF', 'XGB', 'RF', 'kNN', 'SVM-Lin', 'CART', 'LR', 'SVM-Sig', 'ADA'])

In [ ]:
for i in range(10):
    

In [ ]:
    print i